# Session 4 -- Unstructured Files
If you take a look at our data files for Session 4, you will see that our data is not neatly structured as it has been until now. Unfortunately, this is very common. But fear not, as we can programmatically handle these as shape the data to be nicely structured.

# Exercise 1
In Exercise 1, we will only be dealing with metadata. Our goal will be to extract the metadata that resides above the data table, and just for some added utility, we will add in some of our own user-set metadata as well as some calculations.  
![Exercise 1](img/04_exercise1goal.png)

First we'll import packages. We will need `pandas` and `os` as usual, and as mentioned above, we'll do some calculations, so we will need the `math` package as well:

In [ ]:
import os
import pandas as pd
import math

We'll just deal with one input file first. Let's get the path to one of the files in our sample data for Exercise 1:

In [ ]:
infile = r"C:\Users\161289\Py-R\data\session4\exercise1\111ALGN.CSV"

Next let's read that data into a `DataFrame`:

In [ ]:
df0 = pd.read_csv(infile)

You'll see that our `DataFrame` doesn't quite look right...

In [ ]:
df0

So let's deal with that data at the top first. It looks like the first 6 rows is metadata, so let's get those 6 rows first. Notice that these aren't in a format where the first row is column names with data beneath it, and since `read_csv` defaults that first row to column headers, we can specify that we do not have a header row to read:

In [ ]:
df0 = pd.read_csv(infile,header=None,nrows=6)
df0

See now that the column names are just numbers now.

There's a couple of ways to extract the data here. My preferred way is to extract the desired data using the locations in the `DataFrame` for maximum readability. We can use this using the `iloc[row,column]` function. I like to store these in a `dict`ionary just to keep the data organized and in its own structure, so we will first create an empty `dict`:

In [ ]:
metadata = {}
metadata['Date'] = df0.iloc[0,1]
metadata['Time'] = df0.iloc[1,1]
metadata['Wafer ID'] = df0.iloc[2,1]
metadata['Amount'] = df0.iloc[3,1]
metadata['Phase'] = df0.iloc[4,1]
metadata['OrfDir'] = df0.iloc[5,1]
metadata

Since our metadata is so neatly organized, we can do this using a loop as well:

In [ ]:
metadata_loop = {}
for x in range(6):
    metadata_loop[df0.iloc[x,0]] = df0.iloc[x,1]
metadata_loop

Alternatively, we can also transpose the `df0` to match a structured format. We can use the `DataFrame` method `transpose`:

In [ ]:
df1 = df0.transpose()
df1

Now that this `DataFrame` is in the right shape, we can set the column names to match the first row:

In [ ]:
df1.columns = df1.iloc[0]
df1

Now that the headers are correct, we will want to delete that extra first row of "data":

In [ ]:
df1 = df1.drop([0])
df1

And finally, you'll see that the row index on the left is messed up -- that row of data should be row 0. We can fix this by resetting the index with the `reset_index` method. The default behavior of this method is to just slap on a new index column, but by specifying `drop=True` in the parenthesis, we can make it replace the index column that already exists:

In [ ]:
df1 = df1.reset_index(drop=True)
df1

## Note:
Note that in order to do the looping `dict` method and the `transpose` method, the data has to be in a relatively organized format. This is not always the case, so be careful of that.

Let's do a quick show of throwing in some calculations. Before we do the actual math, we have to do some conversions. We'll preserve our original data by creating new key/value pairs and columns.

In the `dict` method, we have `str` types which we have to convert to float:

In [ ]:
type(metadata["Amount"])

In [ ]:
metadata['Amount_num'] = float(metadata['Amount'])
metadata['Phase_num'] = float(metadata['Phase'])
metadata

...and in the `DataFrame` method, we have a `Series` object, which we have to conver to `str` and then to `float`. Since a `Series` is not a built-in type of Python, we have to use the `pandas` conversion method `astype`:

In [ ]:
type(df1['Amount'])

In [ ]:
df1['Amount_num'] = df1['Amount'].astype(str).astype(float)
df1['Phase_num'] = df1['Phase'].astype(str).astype(float)
df1

And finally, let's do the actual conversions. These are some predetermined formulas, so no need to actually know them:

In [ ]:
metadata['RealAngle'] = 360 - metadata['Phase_num']
metadata['x_offset'] = metadata['Amount_num'] * math.cos(metadata['RealAngle']*(math.pi/180))
metadata['y_offset'] = metadata['Amount_num'] * math.sin(metadata['RealAngle']*(math.pi/180))
metadata

Or if using the tranpose version:

In [ ]:
df1['RealAngle'] = 360 - df1['Phase_num']
df1['x_offset'] = df1['Amount_num'] * math.cos(df1['RealAngle']*(math.pi/180))
df1['y_offset'] = df1['Amount_num'] * math.sin(df1['RealAngle']*(math.pi/180)) 
df1

And to get the final output that we saw at the beginning of the exercise, we would just use `insert` commands on some user-defined metadata to tack onto `df1`, or in the case of the `dict` method, `pandas` has a function to convert a `dict` into a `DataFrame` format. We'll skip demonstrating that for now and go straight into the next level -- going through a folder of files and extracting and making calculations on each file using a loop.

# Exercise 1
Let's now get the metadata from each file in the Exercise 1 data folder and compile it into a sort of index file, with the metadata from each file on one line of our output file.

Import packages -- still just `pandas`, `os`, and `math`:

In [ ]:
import os
import pandas as pd
import math

Get the the input folder and a list of the files inside of it:

In [ ]:
inpath = r"C:\Users\161289\Py-R\data\session4\exercise1"
inpathfiles = os.listdir(inpath)
inpathfiles

Create an empty list to contain all of the data:

In [ ]:
dfs = []

Begin the loop -- let's filter to just get the files with "ALGN" in the name"

In [ ]:
for filename in inpathfiles:
    if "ALGN" in filename:

Get the full file path of each file:

In [ ]:
        filename = inpath + "\\" + filename

Then, we can go through the whole process we went through above for one file:

`dict` method:

In [ ]:
        df = pd.read_csv(filename,header=None,nrows=6)
        metadata = {}
        for x in range(6):
            metadata[df0.iloc[x,0]] = df0.iloc[x,1]
        metadata['Amount_num'] = float(metadata['Amount'])
        metadata['Phase_num'] = float(metadata['Phase'])
        metadata['RealAngle'] = 360 - metadata['Phase_num']
        metadata['x_offset'] = metadata['Amount_num'] * math.cos(metadata['RealAngle']*(math.pi/180))
        metadata['y_offset'] = metadata['Amount_num'] * math.sin(metadata['RealAngle']*(math.pi/180))

Here's that function that is able to create a `DataFrame` from a `dict`:

In [ ]:
        df  = pd.DataFrame([metadata], columns=metadata.keys())

`transpose` method:

In [ ]:
        df = pd.read_csv(filename,header=None,nrows=6)
        df = df.transpose()
        df.columns = df.iloc[0]
        df = df.drop([0])
        df = df.reset_index(drop=True)
        df['Amount_num'] = df['Amount'].astype(str).astype(float)
        df['Phase_num'] = df['Phase'].astype(str).astype(float)
        df['RealAngle'] = 360 - df1['Phase_num']
        df['x_offset'] = df['Amount_num'] * math.cos(df1['RealAngle']*(math.pi/180))
        df['y_offset'] = df['Amount_num'] * math.sin(df1['RealAngle']*(math.pi/180))
        dfs.append(df)

Combine all the collected `DataFrame`s into one:

In [ ]:
result = pd.concat(dfs)

Create some metadata to insert -- let's use "TNS" as an equipment name and "Test" as a test name. Then insert it into our `DataFrame`:

In [ ]:
eqname = "TNS"
testname = "Test"
result.insert(0,"Equipment Name",eqname)
result.insert(1,"Test Name", testname)

And finally, go through the steps to output a file -- create the output path, create folders if necessary, create the file name, and create the file:

In [ ]:
outpath = inpath + r"\output"
if not os.path.exists(outpath):
    os.makedirs(outpath)
outfile = outpath + r"\session4output1.csv"
result.to_csv(outfile,index=False)

The final script should look like the following:

`dict` version:

In [ ]:
# import packages -- os, pandas, and math
import os
import pandas as pd
import math

# %% Loop through folder and combine
# input path
inpath = r"C:\Users\161289\Py-R\data\session4\exercise1"
# list of files
inpathfiles = os.listdir(inpath)

# empty list
dfs = []

# %% loop
for filename in inpathfiles:
    # condition/filter -- match with "ALGN"
    if "ALGN" in filename:
        # append path to filename
        filename = inpath + "\\" + filename
        # read header data of file into dataframe
        df = pd.read_csv(filename,header=None,nrows=6)
        
        # empty dict
        metadata = {}
        # collect data into dict
        for x in range(6):
            metadata[df.iloc[x,0]] = df.iloc[x,1]
            
        # convert data to float
        metadata['Amount_num'] = float(metadata['Amount'])
        metadata['Phase_num'] = float(metadata['Phase'])
        
        # calculations
        metadata['RealAngle'] = 360 - metadata['Phase_num']
        metadata['x_offset'] = metadata['Amount_num'] * math.cos(metadata['RealAngle']*(math.pi/180))
        metadata['y_offset'] = metadata['Amount_num'] * math.sin(metadata['RealAngle']*(math.pi/180))
        
        # insert data into DataFrame
        df  = pd.DataFrame([metadata], columns=metadata.keys())
        
        # add data to list
        dfs.append(df)
        
#%% combine data
result = pd.concat(dfs)

# %% Create metadata variables
# EqName: TNS
# TestName: Test
eqname = "TNS"
testname = "Test"

# insert metadata
df.insert(0,"EqName",eqname)
df.insert(1,"TestName",testname)

# %% output file
# set output directory -- create if non-existent
outpath = inpath + r"\output"
if not os.path.exists(outpath):
    os.makedirs(outpath)
# create full output path
outfile = outpath + r"\session4output1dict.csv"

# create ouput file
result.to_csv(outfile,index=False)

`transpose` version:

In [ ]:
# import packages -- os, pandas, and math
import os
import pandas as pd
import math

# %% Loop through folder and combine
# input path
inpath = r"C:\Users\161289\Py-R\data\session4\exercise1"
# list of files
inpathfiles = os.listdir(inpath)

# empty list
dfs = []

# %% loop
for filename in inpathfiles:
    # condition/filter -- match with "ALGN"
    if "ALGN" in filename:
        # append path to filename
        filename = inpath + "\\" + filename
        # read header data of file into dataframe
        df = pd.read_csv(filename,header=None,nrows=6)
        
        # extract column names
        # transpose dataframe
        df = df.transpose()
        # set first row as column names
        df.columns = df.iloc[0]
        # drop first row
        df = df.drop([0])
        # reset index
        df = df.reset_index(drop=True)
        
        # calculations
        # convert Amount and Phase columns to numeric
        df['Amount_num'] = df['Amount'].astype(str).astype(float)
        df['Phase_num'] = df['Phase'].astype(str).astype(float)
        
        # calculate real angle and offsets
        df['RealAngle'] = 360 - df['Phase_num']
        df['x_offset'] = df['Amount_num'] * math.cos(df['RealAngle']*(math.pi/180))
        df['y_offset'] = df['Amount_num'] * math.sin(df['RealAngle']*(math.pi/180))
        
        # add data to list
        dfs.append(df)
        
#%% combine data
result = pd.concat(dfs)

# %% Create metadata variables
# EqName: TNS
# TestName: Test
eqname = "TNS"
testname = "Test"

# insert metadata
result.insert(0,"EqName",eqname)
result.insert(1,"TestName",testname)

# %% output file
# set output directory -- create if non-existent
outpath = inpath + r"\output"
if not os.path.exists(outpath):
    os.makedirs(outpath)
# create full output path
outfile = outpath + r"\session4output1transpose.csv"
# create ouput file
result.to_csv(outfile,index=False)

# Melting Data
Before proceeding to Exercise 2, we are going to go over the concept called "melting" data, also known as "stacking" data in some applications. This takes a wide dataset and crams it into a vertical stacked format. We'll see how this can be useful as we cover PowerBI in our next session, but essentially it enables data to be easily filtered, like one might use in a pivot table.

For example, a dataset that looks like this:

![Pre-melt](img/04_premelt.png)

...would look like this after being melted:

![Melted](img/04_postmelt.png)

# Exercise 2
In exercise 2, we'll step things up a bit more. If you look at our data in the Exercise 2 folder, you'll see that it looks pretty messy -- metadata at the top, multiple datasets placed next to each other, two rows of column data. We'll be extracting that metadata, melting the data, and placing the metadata alongside the melted datatable.

# Initializing
The first steps should be familiar by now -- import packages, get the input files, and start implementing the loop. We'll filter by "WaferFlow" in this exercise.

In [ ]:
import os
import pandas as pd

inpath = r"C:\Users\161289\Py-R\data\session4\exercise2"
inpathfiles = os.listdir(inpath)

dfs = []

for filename in inpathfiles:
    if "WaferFlow" in filename:
        filenamefull = inpath + "\\" + filename

Next comes the logic we just went through in Exercise 1 to grab the metadata.

`dict` version -- note that we can just as easily insert `dict` data into a `DataFrame` as we can another `DataFrame`, so we will skip the conversion step in this case:

In [ ]:
        df_head = pd.read_csv(filenamefull,header=None,nrows=4)
        metadata = {}
        for x in range(df_head.shape[0]):
            metadata[df_head.iloc[x,0]] = df_head.iloc[x,1]        

`transpose` version:

In [ ]:
        df_head = pd.read_csv(filenamefull,header=None,nrows=4)
        df_head = df_head.transpose()
        df_head.columns = df_head.iloc[0]
        df_head = df_head.drop([0])
        df_head = df_head.reset_index(drop=True)

Next, let's get the rest of the `DataFrame`. We can do this by using `skiprows` rather than `nrows`:

In [ ]:
        df = pd.read_csv(filenamefull,skiprows=5)

Next, let's drop that extra row of units that is under the column header names:

In [ ]:
        df = df.drop([0])

Next, we'll melt the data. `pandas` has a built-in function to do this. `id_vars` is which variables to keep as an index, and the names of the variable and value columns are customizable using `var_name` and `value_name`:

In [ ]:
        stacked = pd.melt(df, id_vars=["sec"], var_name="Variable",value_name="Value")

The only data that we're interested in this file is actually the numeric values. The `step` column is not useful in this case, as all of them are just "PreWait". So let's think of how to just isolate the numbers.

In [ ]:
        pd.to_numeric(stacked["Value"], errors='coerce')

Now that those values are `NaN`, we can get rid of them using the `notnull` function:

In [ ]:
        pd.to_numeric(stacked["Value"], errors='coerce').notnull()

This gives us all the rows not to include in our final cleaned `DataFrame`. So let's create that cleaned `DataFrame`:

In [ ]:
        stacked_clean = stacked[pd.to_numeric(stacked["Value"], errors='coerce').notnull()]

Now that we have our cleaned `DataFrame`, let's insert the metadata. We can do this using basic `insert` statements. Here's the `dict` version:

In [ ]:
        stacked_clean.insert(0, 'FileName', filename)
        stacked_clean.insert(1, 'Base Lot Name', metadata['Base Lot Name'])
        stacked_clean.insert(2, 'Wafer Flow Name', metadata['Wafer Flow Name'])
        stacked_clean.insert(3, 'Recipe Step From', metadata['Recipe Step From'])
        stacked_clean.insert(4, 'Recipe Step To', metadata['Recipe Step To'])

And the `transpose` version:

In [ ]:
        stacked_clean.insert(0, 'FileName', filename)
        stacked_clean.insert(1, 'Base Lot Name', df_head.iloc[0]['Base Lot Name'])
        stacked_clean.insert(2, 'Wafer Flow Name', df_head.iloc[0]['Wafer Flow Name'])
        stacked_clean.insert(3, 'Recipe Step From', df_head.iloc[0]['Recipe Step From'])
        stacked_clean.insert(4, 'Recipe Step To', df_head.iloc[0]['Recipe Step To'])

Alternatively, we can do this programmatically by looping through the `dict` if using the `dict` method:

In [ ]:
        for key in metadata:
            stacked_clean.insert(0,key,metadata[key])

...or if using the `transpose` method, loop through a `list` of the columns of `df_head`:

In [ ]:
        columns = list(df_head.columns)
        for x in range(len(columns)):
            stacked_clean.insert(x, columns[x],df_head.iloc[0][columns[x]])

We also want to insert the file name, which wasn't part of the metadata, so let's insert that as well:

In [ ]:
        stacked_clean.insert(0, 'FileName', filename)

And finally, let's finish the loop by appending our cleaned `DataFrame` to our list:

In [ ]:
        dfs.append(stacked_clean)

The rest should be easy -- combine the list into one `DataFrame`, set an output path, and output to a file:

In [ ]:
result = pd.concat(dfs)

outpath = inpath + "\output"
if not os.path.exists(outpath):
    os.makedirs(outpath)
outfile = outpath + "\\session4output2.csv"

result.to_csv(outfile,index=False)

Our final script looks like the following:

If using the `dict` method:

In [ ]:
# import packages -- os and pandas
import os
import pandas as pd

# %% Loop through folder and combine
# input path
# escape sequences: \newline,\\,\',\",\a,\b,\f,\n,\N,\r,\t,\u,\U,\v,\[0-9],\x
inpath = r"C:\Users\161289\Py-R\data\session4\exercise2"
# list of files
inpathfiles = os.listdir(inpath)

# empty list
dfs = []

# %% loop
for filename in inpathfiles:
    # condition/filter -- match with "WaferFlow"
    if "WaferFlow" in filename:
        # append path to filename
        filenamefull = inpath + "\\" + filename
        
        # get metadata
        df_head = pd.read_csv(filenamefull,header=None,nrows=4)
        metadata = {}
        for x in range(df_head.shape[0]):
            metadata[df_head.iloc[x,0]] = df_head.iloc[x,1]      
        
        # read body of file into dataframe
        df = pd.read_csv(filenamefull,skiprows=5)
        
        # delete units row
        df = df.drop([0])
        
        # EXTRA EXPLANATION: create stacked dataframe -- NEW
        stacked = pd.melt(df, id_vars=["sec"], var_name="Variable",value_name="Value")
        # remove nulls -- NEW
        stacked_clean = stacked[pd.to_numeric(stacked["Value"], errors='coerce').notnull()]
        
        # loop
        for key in metadata:
            stacked_clean.insert(0,key,metadata[key])
            
        # add filename
        stacked_clean.insert(0, 'FileName', filename)
        
        # add data to list
        dfs.append(stacked_clean)
#%% combine data
result = pd.concat(dfs)
# %% output file
# set output directory -- create if non-existent
outpath = inpath + "\output"
if not os.path.exists(outpath):
    os.makedirs(outpath)
# create full output path
outfile = outpath + "\\session4output2dict.csv"
# create ouput file
result.to_csv(outfile,index=False)

And for the `transpose` version:

In [ ]:
# import packages -- os and pandas
import os
import pandas as pd

# %% Loop through folder and combine
# input path
# escape sequences: \newline,\\,\',\",\a,\b,\f,\n,\N,\r,\t,\u,\U,\v,\[0-9],\x
inpath = r"C:\Users\161289\Py-R\data\session4\exercise2"
# list of files
inpathfiles = os.listdir(inpath)

# empty list
dfs = []

# %% loop
for filename in inpathfiles:
    # condition/filter -- match with "WaferFlow"
    if "WaferFlow" in filename:
        # append path to filename
        filenamefull = inpath + "\\" + filename
        # read header data of file into dataframe
        df_head = pd.read_csv(filenamefull,header=None,nrows=4)
        
        # extract column names
        # transpose dataframe
        df_head = df_head.transpose()
        # set first row as column names
        df_head.columns = df_head.iloc[0]
        # drop first row
        df_head = df_head.drop([0])
        # reset index
        df_head = df_head.reset_index(drop=True)
        
        # read body of file into dataframe
        df = pd.read_csv(filenamefull,skiprows=5)
        
        # delete units row
        df = df.drop([0])
        
        # EXTRA EXPLANATION: create stacked dataframe -- NEW
        stacked = pd.melt(df, id_vars=["sec"], var_name="Variable",value_name="Value")
        # remove nulls -- NEW
        stacked_clean = stacked[pd.to_numeric(stacked["Value"], errors='coerce').notnull()]
        
        # loop
        # create list of df_head columns
        columns = list(df_head.columns)
        # loop through columns
        for x in range(len(columns)):
            stacked_clean.insert(x, columns[x],df_head.iloc[0][columns[x]])
            
        # add filename
        stacked_clean.insert(0, 'FileName', filename)
        
        # add data to list
        dfs.append(stacked_clean)
#%% combine data
result = pd.concat(dfs)
# %% output file
# set output directory -- create if non-existent
outpath = inpath + "\output"
if not os.path.exists(outpath):
    os.makedirs(outpath)
# create full output path
outfile = outpath + "\\session4output2transpose.csv"
# create ouput file
result.to_csv(outfile,index=False)

# Wrap Up
And that's it for this session! Now you have the tools to manipulate the most intricate of files to get them into a neat, structured format -- you can read files (and/or pieces of them) into `DataFrames`, access specific locations, transpose data, add/delete/modify columns, perform calculations, and more! It all comes down to thinking through the logic of what pieces of the files to read. And with that, we'll move on to visualizations in our next session.